In [36]:
import json
from datasets import Dataset
import pandas as pd

data = json.load(open("/drive2/kaggle/pii-dd/data/train.json"))

all_urls = []
gt_pairs = []
for d in data:
    for i, (t,l) in enumerate(zip(d["tokens"], d["labels"])):
        if "URL_PERSONAL" in l:
            all_urls.append((d["document"], i, t, l))
            gt_pairs.append((d["document"], i))

df = pd.DataFrame(all_urls, columns=["document", "index", "token", "label"])

display(df)

ds = Dataset.from_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/url_classify/2-d3l.pq")
ds

,document,index,token,label
0,317,611,https://www.jackson.com/list/explorehomepage.htm,B-URL_PERSONAL
1,379,31,https://www.linkedin.com/in/mmartinez,B-URL_PERSONAL
2,472,922,https://youtu.be/rFD2lJuvace,B-URL_PERSONAL
3,1309,497,https://www.hall.biz/wp-contenthome.html,B-URL_PERSONAL
4,1798,292,http://www.burns-lopez.com/categories/appabout...,B-URL_PERSONAL
...,...,...,...,...
106,13103,455,https://hernandez.com/exploremain.html,B-URL_PERSONAL
107,13103,457,https://www.roman.info/list/appindex.html,B-URL_PERSONAL
108,13103,649,https://hernandez.com/exploremain.html,B-URL_PERSONAL
109,13103,651,https://www.roman.info/list/appindex.html,B-URL_PERSONAL


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'doc_id', 'token_id', 'input_ids', 'token_type_ids', 'attention_mask', 'length', 'pred'],
    num_rows: 336
})

In [37]:
for i in [0.1, 0.3, 0.5, 0.6, 0.8]:
    preds = []
    for x in ds:
        if x["pred"] > i:
            preds.append((x["doc_id"], x["token_id"]))
    tp = len(set(gt_pairs).intersection(set(preds)))
    fp = len(set(preds) - set(gt_pairs))
    fn = len(set(gt_pairs) - set(preds))
    print(i, tp, fp, fn, tp/(tp+fp+1e-7), tp/(tp+fn+1e-7))

0.1 109 9 2 0.9237288127765011 0.9819819810973136
0.3 106 5 5 0.9549549540946353 0.9549549540946353
0.5 106 2 5 0.9814814805727023 0.9549549540946353
0.6 95 2 16 0.9793814422892976 0.8558558550848145
0.8 91 0 20 0.9999999989010989 0.8198198190812435


In [35]:
for i in [0.1, 0.3, 0.5, 0.6, 0.8]:
    preds = []
    for x in ds:
        if x["pred"] > i:
            preds.append((x["doc_id"], x["token_id"]))
    tp = len(set(gt_pairs).intersection(set(preds)))
    fp = len(set(preds) - set(gt_pairs))
    fn = len(set(gt_pairs) - set(preds))
    print(i, tp, fp, fn, tp/(tp+fp+1e-7), tp/(tp+fn+1e-7))

0.1 107 31 4 0.775362318278723 0.963963963095528
0.3 104 16 7 0.8666666659444445 0.9369369360928497
0.5 95 11 16 0.8962264142488431 0.8558558550848145
0.6 73 1 38 0.9864864851533968 0.6576576570651733
0.8 0 0 111 0.0 0.0


In [33]:
for i in [0.1, 0.3, 0.5, 0.6, 0.8]:
    preds = []
    for x in ds:
        if x["pred"] > i:
            preds.append((x["doc_id"], x["token_id"]))
    tp = len(set(gt_pairs).intersection(set(preds)))
    fp = len(set(preds) - set(gt_pairs))
    fn = len(set(gt_pairs) - set(preds))
    print(i, tp, fp, fn, tp/(tp+fp+1e-7), tp/(tp+fn+1e-7))

0.1 100 14 11 0.8771929816866729 0.9009009000892785
0.3 96 6 15 0.9411764696655133 0.8648648640857074
0.5 91 3 20 0.968085105353101 0.8198198190812435
0.6 90 2 21 0.9782608685018904 0.8108108100803507
0.8 81 1 30 0.9878048768441404 0.7297297290723156


In [38]:
preds = []

for x in ds:
    if x["pred"] > 0.1:
        d = [d for d in data if d["document"] == x["doc_id"]][0]
        print(d["document"], d["tokens"][x["token_id"]], d["labels"][x["token_id"]])

11326 http://hogan.com/app/categoriesindex.php B-URL_PERSONAL
9801 https://www.brooks.net/exploreprivacy.html B-URL_PERSONAL
6853 https://www.linkedin.com/in/michael16 B-URL_PERSONAL
5923 https://jones-hernandez.com/main/tag/listindex.htm B-URL_PERSONAL
6754 https://www.hart.com/posts/tagspost.htm B-URL_PERSONAL
9470 http://perry-king.biz/main/exploreregister.jsp B-URL_PERSONAL
13315 https://alvarado.com/categoriesindex.html B-URL_PERSONAL
1798 http://www.burns-lopez.com/categories/appabout.asp B-URL_PERSONAL
5662 http://chang.com/tagspost.asp B-URL_PERSONAL
7733 https://www.cuevas.info/blogsearch.html B-URL_PERSONAL
11849 http://www.daniels.biz/tagssearch.php B-URL_PERSONAL
11883 http://www.stokes.com/wp-content/main/searchregister.htm B-URL_PERSONAL
7308 https://www.facebook.com/amanda37 B-URL_PERSONAL
10693 https://butler.com/search/wp-content/categorypost.jsp B-URL_PERSONAL
8491 https://www.fitzgerald-nicholson.com/search/categories/wp-contentlogin.htm B-URL_PERSONAL
10945 https://

In [12]:
ds["pred"]

[0.03634050488471985,
 0.022890960797667503,
 0.026120468974113464,
 0.6377571821212769,
 0.024392148479819298,
 0.6198919415473938,
 0.3689489960670471,
 0.019482893869280815,
 0.6332404613494873,
 0.6213624477386475,
 0.6166809797286987,
 0.62214595079422,
 0.020968010649085045,
 0.6077278852462769,
 0.6126059293746948,
 0.6092876195907593,
 0.022892706096172333,
 0.6542432308197021,
 0.019846530631184578,
 0.6154247522354126,
 0.023445360362529755,
 0.6219182014465332,
 0.534250020980835,
 0.02228391543030739,
 0.019892137497663498,
 0.6617323160171509,
 0.6209510564804077,
 0.03069465234875679,
 0.6056582927703857,
 0.6186163425445557,
 0.02343350276350975,
 0.029432449489831924,
 0.022432781755924225,
 0.6416499614715576,
 0.02611633948981762,
 0.02338043600320816,
 0.06665349006652832,
 0.6280224323272705,
 0.6457660794258118,
 0.10141254961490631,
 0.018639646470546722,
 0.02477002516388893,
 0.6488189697265625,
 0.027695685625076294,
 0.031968846917152405,
 0.38016122579574585,

In [6]:
from collections import Counter
Counter([x["pred"] for x in ds])

Counter({0: 336})